In [21]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve
)
import matplotlib.pyplot as plt
import warnings

# Set the display options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 500)
pd.set_option('display.max_colwidth', 500)

student_data = pd.read_csv("/Users/srisatyaindukuri/Spider/ML - Curricular Analytics - PIDM ONLY & Fixed Repeat IND.csv")
# Replace '-' with NaN
student_data.replace('-', np.nan, inplace=True)
student_data.sample(5)


/var/folders/1_/f10h6scn5x19_qnf0xw8fpyr0000gn/T/ipykernel_33198/1500055044.py:18: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  student_data = pd.read_csv("/Users/srisatyaindukuri/Spider/ML - Curricular Analytics - PIDM ONLY & Fixed Repeat IND.csv")


,Pidm,Admit_Code,Admit_Desc,Admit_Term,Admit_Level,Admit_College,Admit_Major_Code,Major_Desc,Trump_Race,Trump_Race_Desc,MULTI,Race,NEW_ETHNICITY,GENDER_Code,GENDER,CITZ_IND,CITZ_CODE,CITZ_DESC,Final_GPA,ACTE,ACTM,ACTR,ACTS,EACT,SAT-ERW,SATM,SAT_TOTAL,Term,CRN,SUBJ,CRSE_NUMB,REPEAT_IND,FINAL_GRADE
2077729,3579950,BL,Beginner(FTIC) - Lower Level,202005,UG,SA,BEC,Early Childhood Education,B,Black or African American,NaN,B Black or African American,Not Hispanic or Latino,F,Female,Y,C,Citizen (USA),3.25,NaN,NaN,NaN,NaN,NaN,670,550,1220,202005.0,58978.0,EDF,2005,NaN,A
1517352,3429399,BL,Beginner(FTIC) - Lower Level,201805,UG,SA,EHN,Exploratory-Health&Natural Sci,U,Unknown,NaN,NaN,Not Hispanic or Latino,F,Female,Y,C,Citizen (USA),4.51,20,18,25,24,22,610,560,1170,202101.0,16644.0,PHC,4720,NaN,A
313457,3092673,BL,Beginner(FTIC) - Lower Level,201408,UG,SA,PSY,Psychology,H,Hispanic or Latino,NaN,W White,Hispanic or Latino,F,Female,Y,C,Citizen (USA),4.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,201508.0,82932.0,PGY,2401C,NaN,B
408235,3118846,BL,Beginner(FTIC) - Lower Level,201408,UG,SP,BIO,Biology,W,White,NaN,W White,Not Hispanic or Latino,M,Male,Y,C,Citizen (USA),2.95,26,25,27,22,25,NaN,NaN,NaN,201408.0,89528.0,MAC,1147,E,D+
2430032,3706465,JU,Fl Comm Coll Transf-Upper Levl,202301,UG,BC,SCW,Social Work,H,Hispanic or Latino,NaN,NaN,Hispanic or Latino,F,Female,Y,C,Citizen (USA),3.94,NaN,NaN,NaN,NaN,NaN,490,510,1000,202308.0,83418.0,SOW,4341,NaN,W


In [22]:
# List of score columns
score_columns = ['ACTE', 'ACTM', 'ACTR', 'ACTS', 'EACT', 'SAT-ERW', 'SATM', 'SAT_TOTAL', 'Final_GPA']

# Convert score columns to numeric
for col in score_columns:
    student_data[col] = pd.to_numeric(student_data[col], errors='coerce')

In [24]:
# ACT to SAT conversion table
act_to_sat_conversion = {
    36: 1590, 35: 1540, 34: 1500, 33: 1460, 32: 1430, 31: 1400,
    30: 1370, 29: 1340, 28: 1310, 27: 1280, 26: 1240, 25: 1210,
    24: 1180, 23: 1140, 22: 1110, 21: 1080, 20: 1040, 19: 1010,
    18: 970, 17: 930, 16: 890, 15: 850, 14: 800, 13: 760,
    12: 710, 11: 670, 10: 630, 9: 590
}

# Convert EACT to SAT
def convert_act_to_sat(eact_score):
    if pd.isna(eact_score):
        return np.nan
    return act_to_sat_conversion.get(int(eact_score), np.nan)

# Apply the conversion to EACT scores
student_data['Converted_SAT'] = student_data['SAT_TOTAL']

# Identify where SAT_TOTAL is missing but EACT is available
mask = student_data['SAT_TOTAL'].isna() & student_data['EACT'].notna()

# Apply conversion
student_data.loc[mask, 'Converted_SAT'] = student_data.loc[mask, 'EACT'].apply(convert_act_to_sat)

# Step 4: Handle Remaining Missing Values

# Drop rows where Converted_SAT or Final_GPA is still NaN
datawithsat = student_data.dropna(subset=['Converted_SAT', 'Final_GPA'])

datawithsat.sample(5)

,Pidm,Admit_Code,Admit_Desc,Admit_Term,Admit_Level,Admit_College,Admit_Major_Code,Major_Desc,Trump_Race,Trump_Race_Desc,MULTI,Race,NEW_ETHNICITY,GENDER_Code,GENDER,CITZ_IND,CITZ_CODE,CITZ_DESC,Final_GPA,ACTE,ACTM,ACTR,ACTS,EACT,SAT-ERW,SATM,SAT_TOTAL,Term,CRN,SUBJ,CRSE_NUMB,REPEAT_IND,FINAL_GRADE,Converted_SAT
815264,3249409,UU,Other UG Transf - Upper Level,202005,UG,SA,PHY,Physics,B,Black or African American,NaN,B Black or African American,Not Hispanic or Latino,M,Male,Y,C,Citizen (USA),3.55,23.0,19.0,23.0,17.0,21.0,560.0,530.0,1090.0,202301.0,10880.0,PHY,4523,NaN,C,1090.0
1858555,3521550,JL,Fl Comm Coll Transf-Lower Levl,202108,UG,SA,PSY,Psychology,A,Asian,NaN,A Asian,Not Hispanic or Latino,M,Male,Y,C,Citizen (USA),2.50,18.0,16.0,22.0,18.0,19.0,NaN,NaN,NaN,202108.0,95343.0,QMB,2100,NaN,C+,1010.0
866710,3263536,BL,Beginner(FTIC) - Lower Level,201605,UG,SA,PMS,Pre-Medical Sciences,B,Black or African American,NaN,B Black or African American,Not Hispanic or Latino,F,Female,Y,C,Citizen (USA),3.80,24.0,23.0,23.0,23.0,23.0,NaN,NaN,NaN,201701.0,12521.0,MAC,2241,NaN,W,1140.0
2221932,3632610,BL,Beginner(FTIC) - Lower Level,202201,UG,PH,HSS,Health Sciences,B,Black or African American,Multi-Race,B Black or African American\nW White,Not Hispanic or Latino,F,Female,Y,C,Citizen (USA),4.27,22.0,17.0,22.0,20.0,20.0,630.0,530.0,1160.0,202208.0,91392.0,ANT,3101,NaN,A,1160.0
2441186,3710814,BL,Beginner(FTIC) - Lower Level,202208,UG,AC,BMS,Biomedical Sciences,A,Asian,NaN,A Asian,Not Hispanic or Latino,F,Female,Y,C,Citizen (USA),4.28,NaN,NaN,NaN,NaN,NaN,600.0,700.0,1300.0,202208.0,88068.0,IDH,2010,NaN,B,1300.0


In [25]:
grade_mapping = {
    # Standard Letter Grades
    'A': 4.0,
    'A+': 4.0,
    'A-': 3.67,  # Note: Some entries list A- as 3.67
    'B+': 3.5,   # Note: Some entries list B+ as 3.33
    'B': 3.0,
    'B-': 2.75,  # Note: Some entries list B- as 2.67
    'C+': 2.5,   # Note: Some entries list C+ as 2.33
    'C': 2.0,
    'C-': 1.75,  # Note: Some entries list C- as 1.67
    'D+': 1.5,   # Note: Some entries list D+ as 1.33
    'D': 1.0,
    'D-': 0.75,  # Note: Some entries list D- as 0.67
    'F': 0.0,
    'FF': 0.0,   # Dishonesty Failing Grade
    'MF': 0.0,   # Missing Failing Grade
    'IF': 0.0,   # Incomplete Failing Grade

    # Satisfactory/Unsatisfactory Grades
    'S': 0.0,    # In some systems, 'S' does not affect GPA
    'U': 0.0,

    # Pass/Fail Grades
    'P': 0.0,    # Pass (may not affect GPA)
    'NP': 0.0,   # No Pass

    # Withdrawal Grades
    'W': 0.0,
    'WE': 0.0,
    'WC': 0.0,
    'WF': 0.0,   # Withdraw Failing (may affect GPA)
    'WP': 4.0,  # Withdraw Passing

    # Incomplete Grades
    'I': None,
    'IA': None,
    'IA+': None,
    'IA-': None,
    'IB': None,
    'IB+': None,
    'IB-': None,
    'IC': None,
    'IC+': None,
    'IC-': None,
    'ID': None,
    'ID+': None,
    'ID-': None,
    'IP': None,
    'IS': None,
    'IU': None,
    'IX': None,
    'M': None,   # Missing Grade
    'MG': None,  # Missing Grade

    # Audit Grades
    'AU': None,
    'N': None,   # Audit or No Grade

    # Not Reported/No Grade
    'NR': None,
    'NG': None,  # No Grade
    'NC': None,  # No Credit

    # Administrative Grades
    'Z': None,   # Continuing Course
    'X': None,   # Deferred Grade

    # Special Cases and Other Grades
    'R': None,   # Repeat
    'IP': None,  # In Progress
    'S*': 0.0,
    'U*': 0.0,
    'W*': None,
    'I*': None,
    'Z*': None,
    'M*': None,

    # Grades with Special Characters Cleaned
    'A*': 4.0,
    'A#': 0.0,
    'A+*': 4.0,
    'A+*#': 0.0,
    'A-*': 3.75,
    'A-#': 0.0,
    'B*': 3.0,
    'B#': 0.0,
    'B+*': 3.5,
    'B+#': 0.0,
    'B-*': 2.75,
    'B-#': 0.0,
    'C*': 2.0,
    'C#': 0.0,
    'C+*': 2.5,
    'C+#': 0.0,
    'C-*': 1.75,
    'C-#': 0.0,
    'D*': 1.0,
    'D#': 0.0,
    'D+*': 1.5,
    'D+#': 0.0,
    'D-*': 0.75,
    'D-#': 0.0,
    'F*': 0.0,
    'F#': 0.0,
    'IF*': 0.0,
    'MF*': 0.0,

    # Competency-Based Grades (May not affect GPA)
    'CA+': None,
    'CA': None,
    'CA-': None,
    'CB': None,
    'CB+': None,
    'CB-': None,
    'CC': None,
    'CC+': None,
    'CC-': None,
    'CD': None,
    'CD+': None,
    'CD-': None,
    'CF': None,
    'CFF': None,
    'CI': None,
    'CIF': None,
    'CIS': None,
    'CM': None,
    'CMF': None,
    'CMU': None,
    'CN': None,
    'CS': None,
    'CU': None,
    'CW': None,
    'CX': None,
    'CZ': None,

    # Q Grades (Special Grading Scale)
    'Q1': 1.5,   # Equivalent to D+
    'Q2': 1.75,  # Equivalent to C-
    'Q3': 2.5,   # Equivalent to C+
    'Q4': 2.75,  # Equivalent to B-
    'Q5': 3.5,   # Equivalent to B+
    'Q6': 3.75,  # Equivalent to A-
    'QA': 4.0,   # Equivalent to A
    'QB': 3.0,   # Equivalent to B
    'QC': 2.0,   # Equivalent to C
    'QD': 1.0,   # Equivalent to D
    'QF': 0.0,   # Equivalent to F

    # Other Grades
    'IP': None,
    'IN': None,
    'NG*': None,
    'NR*': None,
    'S#': 0.0,
    'U#': 0.0,
    'W#': None,
    'X#': None,
    'Z#': None,

    # Repeat Indicators
    'R': None,
}

# Apply the mapping to the FINAL_GRADE column
datawithsat['FINAL_GRADE_NUM'] = datawithsat['FINAL_GRADE'].map(grade_mapping)
# Drop rows where 'FINAL_GRADE_NUM' is NaN
datawithsat = datawithsat.dropna(subset=['FINAL_GRADE_NUM'])

# Print the updated DataFrame
datawithsat.sample(5)

/var/folders/1_/f10h6scn5x19_qnf0xw8fpyr0000gn/T/ipykernel_33198/3412161981.py:166: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datawithsat['FINAL_GRADE_NUM'] = datawithsat['FINAL_GRADE'].map(grade_mapping)


,Pidm,Admit_Code,Admit_Desc,Admit_Term,Admit_Level,Admit_College,Admit_Major_Code,Major_Desc,Trump_Race,Trump_Race_Desc,MULTI,Race,NEW_ETHNICITY,GENDER_Code,GENDER,CITZ_IND,CITZ_CODE,CITZ_DESC,Final_GPA,ACTE,ACTM,ACTR,ACTS,EACT,SAT-ERW,SATM,SAT_TOTAL,Term,CRN,SUBJ,CRSE_NUMB,REPEAT_IND,FINAL_GRADE,Converted_SAT,FINAL_GRADE_NUM
2248224,3637987,BL,Beginner(FTIC) - Lower Level,202108,UG,SA,CCJ,Criminology,W,White,NaN,W White,Not Hispanic or Latino,F,Female,Y,C,Citizen (USA),4.55,32.0,29.0,36.0,33.0,33.0,740.0,730.0,1470.0,202108.0,90811.0,IDH,2930,NaN,S,1470.0,0.00
1851377,3518165,BL,Beginner(FTIC) - Lower Level,201908,UG,SA,HLS,Health Sciences,W,White,NaN,W White,Not Hispanic or Latino,M,Male,Y,C,Citizen (USA),3.46,23.0,24.0,19.0,21.0,22.0,570.0,640.0,1210.0,201908.0,80372.0,MAC,1105,NaN,A,1210.0,4.00
2585445,3785408,UL,Other UG Transf - Lower Level,202208,UG,EN,MNE,Mechanical Engineering,B,Black or African American,Multi-Race,B Black or African American\nW White,Not Hispanic or Latino,M,Male,Y,C,Citizen (USA),4.18,NaN,NaN,NaN,NaN,NaN,580.0,720.0,1300.0,202308.0,86081.0,NSC,1101L,I,S,1300.0,0.00
2135107,3599857,JU,Fl Comm Coll Transf-Upper Levl,202008,UG,SA,EPME,Pre-Mechanical Engineering,W,White,NaN,W White,Not Hispanic or Latino,M,Male,Y,C,Citizen (USA),4.02,32.0,30.0,34.0,36.0,33.0,760.0,700.0,1460.0,202008.0,88187.0,EGN,3000,NaN,S,1460.0,0.00
1377032,3402591,FS,Former USF Student Returning,202005,UG,SA,HSS,Health Sciences,W,White,NaN,W White,Not Hispanic or Latino,F,Female,Y,C,Citizen (USA),4.00,NaN,NaN,NaN,NaN,NaN,590.0,600.0,1190.0,202008.0,92451.0,CLT,3040,NaN,A-,1190.0,3.67


In [26]:
# Group by 'Pidm' (student ID)
grouped = datawithsat.groupby('Pidm')
grouped.head(10)

,Pidm,Admit_Code,Admit_Desc,Admit_Term,Admit_Level,Admit_College,Admit_Major_Code,Major_Desc,Trump_Race,Trump_Race_Desc,MULTI,Race,NEW_ETHNICITY,GENDER_Code,GENDER,CITZ_IND,CITZ_CODE,CITZ_DESC,Final_GPA,ACTE,ACTM,ACTR,ACTS,EACT,SAT-ERW,SATM,SAT_TOTAL,Term,CRN,SUBJ,CRSE_NUMB,REPEAT_IND,FINAL_GRADE,Converted_SAT,FINAL_GRADE_NUM
15,794,SB,Second Baccalaureate,201508,UG,SA,PNR,Pre-Nursing,W,White,NaN,W White,Not Hispanic or Latino,M,Male,Y,C,Citizen (USA),3.30,25.0,26.0,25.0,22.0,25.0,NaN,NaN,NaN,201508.0,83267.0,NUR,3805,NaN,A+,1210.0,4.00
16,794,SB,Second Baccalaureate,201508,UG,SA,PNR,Pre-Nursing,W,White,NaN,W White,Not Hispanic or Latino,M,Male,Y,C,Citizen (USA),3.30,25.0,26.0,25.0,22.0,25.0,NaN,NaN,NaN,201508.0,89943.0,NUR,3078,NaN,A+,1210.0,4.00
17,794,SB,Second Baccalaureate,201508,UG,SA,PNR,Pre-Nursing,W,White,NaN,W White,Not Hispanic or Latino,M,Male,Y,C,Citizen (USA),3.30,25.0,26.0,25.0,22.0,25.0,NaN,NaN,NaN,201508.0,94441.0,NUR,4895,NaN,A+,1210.0,4.00
18,794,SB,Second Baccalaureate,201508,UG,SA,PNR,Pre-Nursing,W,White,NaN,W White,Not Hispanic or Latino,M,Male,Y,C,Citizen (USA),3.30,25.0,26.0,25.0,22.0,25.0,NaN,NaN,NaN,201601.0,22277.0,NUR,4828C,NaN,A+,1210.0,4.00
19,794,SB,Second Baccalaureate,201508,UG,SA,PNR,Pre-Nursing,W,White,NaN,W White,Not Hispanic or Latino,M,Male,Y,C,Citizen (USA),3.30,25.0,26.0,25.0,22.0,25.0,NaN,NaN,NaN,201601.0,22671.0,NUR,4128,NaN,A+,1210.0,4.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2677342,3984725,UL,Other UG Transf - Lower Level,202405,UG,PH,HSS,Health Sciences,W,White,NaN,W White,Not Hispanic or Latino,F,Female,Y,C,Citizen (USA),3.64,NaN,NaN,NaN,NaN,NaN,580.0,590.0,1170.0,202405.0,55256.0,SPC,2608,NaN,A-,1170.0,3.67
2677343,3984750,UL,Other UG Transf - Lower Level,202405,UG,BU,PAIS,Pre-Bus Analytics & Info Sys,W,White,NaN,W White,Not Hispanic or Latino,M,Male,Y,C,Citizen (USA),3.29,18.0,17.0,18.0,20.0,18.0,410.0,340.0,750.0,202405.0,51968.0,MMC,3602,NaN,C+,750.0,2.50
2677344,3984772,BL,Beginner(FTIC) - Lower Level,202405,UG,AC,PSY,Psychology,F,Non-Resident Alien,NaN,NaN,NaN,F,Female,N,F,Foreign National,0.00,NaN,NaN,NaN,NaN,NaN,450.0,480.0,930.0,202405.0,54569.0,HUM,1020,NaN,F,930.0,0.00
2677400,3988409,UL,Other UG Transf - Lower Level,202405,UG,AC,PSY,Psychology,W,White,NaN,W White,Not Hispanic or Latino,M,Male,Y,C,Citizen (USA),3.73,NaN,NaN,NaN,NaN,NaN,560.0,560.0,1120.0,202405.0,51082.0,PSY,2012,NaN,B,1120.0,3.00


In [29]:
all_data = []
# Process each student
for pidm, student_data in grouped:
    # Sort the student's data by 'Term'
    student_data = student_data.sort_values('Term')
    # Reset index to ensure correct iteration
    student_data = student_data.reset_index(drop=True)
    # Initialize an empty list to keep track of cumulative grades
    cumulative_grades = []
    for idx, row in student_data.iterrows():
        current_term = row['Term']
        # Get previous classes before the current term
        previous_classes = student_data[student_data['Term'] < current_term]
        # Compute cumulative GPA up to the current term
        if not previous_classes.empty:
            cumulative_gpa = previous_classes['FINAL_GRADE_NUM'].mean()
        else:
            cumulative_gpa = None  # No previous grades
        # Collect previous classes data
        previous_classes_list = previous_classes[['SUBJ', 'CRSE_NUMB', 'FINAL_GRADE', 'FINAL_GRADE_NUM']].to_dict('records')
        # Create a new row with current class data and previous classes info
        new_row = row.to_dict()
        new_row['Previous_Classes'] = previous_classes_list
        new_row['Cumulative_GPA'] = cumulative_gpa
        all_data.append(new_row)

# Create a new DataFrame from the list
new_df = pd.DataFrame(all_data)

# Display the new DataFrame
#print(new_df[['Pidm', 'Term', 'SUBJ', 'CRSE_NUMB', 'FINAL_GRADE', 'Cumulative_GPA', 'Previous_Classes']])
new_df.sample(5)

,Pidm,Admit_Code,Admit_Desc,Admit_Term,Admit_Level,Admit_College,Admit_Major_Code,Major_Desc,Trump_Race,Trump_Race_Desc,MULTI,Race,NEW_ETHNICITY,GENDER_Code,GENDER,CITZ_IND,CITZ_CODE,CITZ_DESC,Final_GPA,ACTE,ACTM,ACTR,ACTS,EACT,SAT-ERW,SATM,SAT_TOTAL,Term,CRN,SUBJ,CRSE_NUMB,REPEAT_IND,FINAL_GRADE,Converted_SAT,FINAL_GRADE_NUM,Previous_Classes,Cumulative_GPA
989096,3360405,BL,Beginner(FTIC) - Lower Level,201708,UG,SP,ESP,Environmental Science & Policy,W,White,NaN,W White,Not Hispanic or Latino,M,Male,Y,C,Citizen (USA),3.96,23.0,25.0,33.0,24.0,26.0,NaN,NaN,NaN,201908.0,94150.0,BOT,4714C,NaN,A-,1240.0,3.67,"[{'SUBJ': 'EVR', 'CRSE_NUMB': '2001', 'FINAL_GRADE': 'A', 'FINAL_GRADE_NUM': 4.0}, {'SUBJ': 'EVR', 'CRSE_NUMB': '2001L', 'FINAL_GRADE': 'B', 'FINAL_GRADE_NUM': 3.0}, {'SUBJ': 'BSC', 'CRSE_NUMB': '2010L', 'FINAL_GRADE': 'A', 'FINAL_GRADE_NUM': 4.0}, {'SUBJ': 'STA', 'CRSE_NUMB': '2023', 'FINAL_GRADE': 'C', 'FINAL_GRADE_NUM': 2.0}, {'SUBJ': 'BSC', 'CRSE_NUMB': '2010', 'FINAL_GRADE': 'B', 'FINAL_GRADE_NUM': 3.0}, {'SUBJ': 'EVR', 'CRSE_NUMB': '2861', 'FINAL_GRADE': 'B', 'FINAL_GRADE_NUM': 3.0}, {...",3.506429
1028970,3376814,JL,Fl Comm Coll Transf-Lower Levl,201705,UG,SA,HTY,History,W,White,NaN,W White,Not Hispanic or Latino,M,Male,Y,C,Citizen (USA),4.05,NaN,NaN,NaN,NaN,NaN,500.0,460.0,960.0,201801.0,10046.0,CHM,2045L,NaN,A+,960.0,4.00,"[{'SUBJ': 'GLY', 'CRSE_NUMB': '2100', 'FINAL_GRADE': 'A', 'FINAL_GRADE_NUM': 4.0}, {'SUBJ': 'CHM', 'CRSE_NUMB': '2045', 'FINAL_GRADE': 'A+', 'FINAL_GRADE_NUM': 4.0}, {'SUBJ': 'GLY', 'CRSE_NUMB': '2010', 'FINAL_GRADE': 'A', 'FINAL_GRADE_NUM': 4.0}, {'SUBJ': 'MAC', 'CRSE_NUMB': '1147', 'FINAL_GRADE': 'A', 'FINAL_GRADE_NUM': 4.0}, {'SUBJ': 'GLY', 'CRSE_NUMB': '2000L', 'FINAL_GRADE': 'A-', 'FINAL_GRADE_NUM': 3.67}, {'SUBJ': 'GEO', 'CRSE_NUMB': '4284', 'FINAL_GRADE': 'A+', 'FINAL_GRADE_NUM': 4.0}]",3.945000
1469247,3499022,UU,Other UG Transf - Upper Level,202108,UG,SA,CHM,Chemistry,H,Hispanic or Latino,NaN,W White,Hispanic or Latino,M,Male,Y,C,Citizen (USA),3.72,NaN,NaN,NaN,NaN,NaN,550.0,520.0,1070.0,202301.0,14849.0,MCB,3410,NaN,B,1070.0,3.00,"[{'SUBJ': 'PCB', 'CRSE_NUMB': '3043', 'FINAL_GRADE': 'A', 'FINAL_GRADE_NUM': 4.0}, {'SUBJ': 'GEY', 'CRSE_NUMB': '3601', 'FINAL_GRADE': 'A', 'FINAL_GRADE_NUM': 4.0}, {'SUBJ': 'PCB', 'CRSE_NUMB': '3043L', 'FINAL_GRADE': 'A', 'FINAL_GRADE_NUM': 4.0}, {'SUBJ': 'CHM', 'CRSE_NUMB': '2210', 'FINAL_GRADE': 'B', 'FINAL_GRADE_NUM': 3.0}, {'SUBJ': 'MHS', 'CRSE_NUMB': '3063', 'FINAL_GRADE': 'A', 'FINAL_GRADE_NUM': 4.0}, {'SUBJ': 'CHM', 'CRSE_NUMB': '2211', 'FINAL_GRADE': 'B', 'FINAL_GRADE_NUM': 3.0}, {'...",3.735294
1007724,3367160,BL,Beginner(FTIC) - Lower Level,201708,UG,SM,BSA,Pre-Business Administration,W,White,NaN,W White,Not Hispanic or Latino,M,Male,Y,C,Citizen (USA),4.08,21.0,19.0,22.0,19.0,20.0,540.0,530.0,1070.0,201708.0,95588.0,ENC,1101,NaN,A+,1070.0,4.00,[],NaN
467492,3191523,BL,Beginner(FTIC) - Lower Level,201508,UG,SP,DUS,Undeclared Major,W,White,NaN,W White,Not Hispanic or Latino,F,Female,Y,C,Citizen (USA),3.66,23.0,19.0,27.0,17.0,22.0,NaN,NaN,NaN,201605.0,51488.0,CGS,2100,NaN,B,1110.0,3.00,"[{'SUBJ': 'ECO', 'CRSE_NUMB': '2013', 'FINAL_GRADE': 'B', 'FINAL_GRADE_NUM': 3.0}, {'SUBJ': 'ENC', 'CRSE_NUMB': '1101', 'FINAL_GRADE': 'B', 'FINAL_GRADE_NUM': 3.0}, {'SUBJ': 'MAT', 'CRSE_NUMB': '1033', 'FINAL_GRADE': 'D', 'FINAL_GRADE_NUM': 1.0}, {'SUBJ': 'EDG', 'CRSE_NUMB': '2930', 'FINAL_GRADE': 'A', 'FINAL_GRADE_NUM': 4.0}, {'SUBJ': 'ESC', 'CRSE_NUMB': '2000', 'FINAL_GRADE': 'A', 'FINAL_GRADE_NUM': 4.0}, {'SUBJ': 'SPC', 'CRSE_NUMB': '2608', 'FINAL_GRADE': 'A-', 'FINAL_GRADE_NUM': 3.67}, {...",3.134000


In [32]:
new_df.to_csv('new_df.csv', index=False)